Notwendige Pakete

In [15]:
### Ziel: Installieren der notwendigen Pakete und Einrichtung der Umgebung ###

#Empfehlung: Neue Conda-Umgebung einrichten im Terminal
#!conda create --name beng python=3.12.0
#!conda activate beng

#%pip install requests
#%pip install xml.etree.ElementTree
#%pip install --upgrade --quiet langchain langchain-openai
#%pip install python-dotenv
#%pip install translate
#%pip install reportlab
#%pip install pdfkit
#%pip install lxml



Zugriff DEXPI-R&I 

In [16]:
### Ziel: Zugriff auf das DEXPI-R&I im .xml-Format herstellen ###
# Quelle GitHub Repositry der Bachelorarbeit: https://github.com/FloT10/MillerFlorinBA

# Importieren des 'requests'-Moduls, um Daten von einer URL herunterzuladen, und das 'xml.etree.ElementTree'-Modul, um XML-Daten zu verarbeiten.
import requests
import xml.etree.ElementTree as ET

# GitHub Raw-URLs für die DEXPI-Datei.
github_url_dexpi = 'https://raw.githubusercontent.com/FloT10/MillerFlorinBA/main/C01V01-HEX.EX03.xml'
# Im aktuellen Entwurf nicht nötig.
# github_url_formblatt = 'https://github.com/FloT10/MillerFlorinBA/blob/546c3688eccc25c23648de29965ede3a476fcb0b/formblatt_6_2.xml'

# Herunterladen der DEXPI-Datei von der GitHub-URL.
response = requests.get(github_url_dexpi)

# Speichern der heruntergeladenen DEXPI-Datei lokal im Textmodus ('w').
with open('/tmp/C01V01-HEX.EX03.xml', 'w') as file:
    file.write(response.text)

# Definieren eines Dateipfades zur lokal gespeicherten DEXPI-Datei zur weiteren Verwendung.
dexpi_xml = "/tmp/C01V01-HEX.EX03.xml"

with open('/tmp/C01V01-HEX.EX03.xml', 'r') as file:
    dexpi_xml_content = file.read(1000)
    print(dexpi_xml_content)

<?xml version="1.0" encoding="utf-8"?>
<PlantModel xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="../ProteusPIDSchema_4.0.1.xsd">
  <!--Created with INGR ISO15926 PostProc V0.0.7.1 and ConfigurationFileVersion 1.1 by PPM-VM-2014-R1\Intergraph on PPM-VM-2014-R1 at 4/12/2018 5:14:33 PM-->
  <PlantInformation SchemaVersion="4.0.1" OriginatingSystem="SPPID" Date="2018-04-12" Time="15:13:38.0000000+02:00" Is3D="no" Units="Metre" Discipline="PID">
    <UnitsOfMeasure Distance="Metre" />
  </PlantInformation>
  <PlantStructureItem ID="PBS945900F29F424D32A292F07BB90792E2" ComponentClass="PlantSectionIso10209-2012" ComponentName="Github" ComponentClassURI="http://sandbox.dexpi.org/rdl/PlantSectionIso10209-2012">
    <GenericAttributes Number="2" Set="DexpiAttributes">
      <GenericAttribute Name="PlantSectionIdentificationCodeAssignmentClass" Value="Github" Format="string" AttributeURI="http://sandbox.dexpi.org/rdl/PlantSectionIdentificationCodeAssignment


Angaben zur Anlage

In [17]:
### Ziel: Extrahieren der Anlagenbezeichnung ###

# Globale Variable für Anlagenbezeichnung
anlagenbezeichnung = None

# Funktion zum Laden und Extrahieren von Daten aus der DEXPI-Datei.
def lade_daten(dexpi_xml):
    global anlagenbezeichnung
    try:
        # Laden des XML-Dokuments.
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahiere die Anlagenbezeichnung aus dem XML-Baum.
        drawing_element = root.find('.//Drawing')
        if drawing_element is not None:
            anlagenbezeichnung = drawing_element.get('Name')

        else:
            print("Die Anlagenbezeichnung wurde nicht gefunden.")

    except Exception as e:
        # Falls ein Fehler auftritt, gib eine Fehlermeldung aus.
        print(f"Fehler beim Laden und Extrahieren der Daten: {e}")

# Beispielaufruf der Funktion
lade_daten(dexpi_xml)

# Verwende die globale Variable an anderer Stelle im Skript, wenn nötig.
print(anlagenbezeichnung)


C01V01-HEX.EX03


In [18]:
### Ziel: Extrahieren des Anlagenumfangs ###

# Funktion zum Laden und Extrahieren von Daten aus der DEXPI-Datei.
def lade_daten(dexpi_xml):
    try:
        # Lade das XML-Dokument und erhalte die Wurzel des Baums.
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahiere ComponentClass aus allen Equipment- und PipingNetworkSystem-Elementen.
        equipment_elements = root.findall('.//Equipment')
        piping_network_system_elements = root.findall('.//PipingNetworkSystem')

        component_classes = set()

        for equipment_element in equipment_elements:
            component_class = equipment_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            # Ausgerichtet am Beispiel-RI. Andere Anlagenbestandteile könnten ergänzt werden.
            erlaubte_component_classes = ['Tank', 'ReciprocatingPump', 'CentrifugalPump', 'PlateAndShellHeatExchanger', 'ShellAndTubeHeatExchanger']
            if component_class in erlaubte_component_classes:
                component_classes.add(component_class)

        for piping_network_system_element in piping_network_system_elements:
            component_class = piping_network_system_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            erlaubte_component_classes = ['PipingNetworkSystem']
            if component_class in erlaubte_component_classes:
                component_classes.add(component_class)

        # Überprüfe, ob mindestens eine gültige ComponentClass gefunden wurde.
        if not component_classes:
            raise ValueError("Keine gültigen Anlagenbestandteile gefunden.")

        # Weitere Werte können entsprechend extrahiert werden.

        # Gib ein Dictionary mit den extrahierten Werten zurück.
        return {
            'Anlagenumfang (englisch)': list(component_classes),
            # Weitere extrahierte Werte könnten hier hinzugefügt werden.
        }

    except Exception as e:
        # Falls ein Fehler auftritt, gib eine Fehlermeldung aus und gibt None zurück.
        print(f"Fehler beim Laden und Extrahieren der Daten: {e}")
        return None

# Beispielaufruf der Funktion und Ausgabe der extrahierten Daten.
anlagenumfang_englisch = lade_daten(dexpi_xml)
# print(anlagenumfang_englisch)

def uebersetze_begriffe_manuell(begriffe, zielsprache='de'):
    manuelle_uebersetzungen = {
        'ReciprocatingPump': 'Kolbenpumpe',
        'PipingNetworkSystem': 'Rohrleitungssystem',
        'CentrifugalPump': 'Zentrifugalpumpe',
        'PlateAndShellHeatExchanger': 'Plattenwärmetauscher',
        'Tank': 'Behälter',
        'ShellAndTubeHeatExchanger': 'Rohrbündelwärmetauscher'
    }

    uebersetzte_begriffe = [manuelle_uebersetzungen.get(begriff, begriff) for begriff in begriffe]
    return uebersetzte_begriffe

# Beispielaufruf
anlagenumfang_liste = uebersetze_begriffe_manuell(anlagenumfang_englisch['Anlagenumfang (englisch)'])

# Verbinden der Elemente der Liste zu einem String mit Kommas
anlagenumfang = ", ".join(anlagenumfang_liste)


# Ausgabe der übersetzten Begriffe
print(anlagenumfang)



Zentrifugalpumpe, Rohrleitungssystem, Kolbenpumpe, Plattenwärmetauscher, Behälter, Rohrbündelwärmetauscher


In [19]:
### Ziel: Extrahieren der Anlagenart ###

def finde_groessten_tank(dexpi_xml):
    try:
        # Parsen der DEXPI-Datei
        root = ET.fromstring(dexpi_xml)
    except ET.ParseError as e:
        # Fehlerausgabe, wenn das Parsen fehlschlägt
        print("Fehler beim Parsen des XML-Dokuments:", e)
        return None, None
    
    # Initialisierung von Variablen für den größten Tank und dessen Kapazität
    groesster_tank = None
    max_capacity = 0
    
    # Iterieren über alle 'Equipment'-Elemente im XML-Dokument
    for tank in root.iter('Equipment'):
        if tank.attrib.get('ComponentClass') == 'Tank':
            # Suche nach dem Attribut 'NominalCapacity(Volume)' im aktuellen Tank
            capacity_attr = tank.find(".//GenericAttribute[@Name='NominalCapacity(Volume)']")
            
            # Überprüfen, ob das Attribut gefunden wurde und einen 'Value'-Schlüssel hat
            if capacity_attr is not None and 'Value' in capacity_attr.attrib:
                # Extrahieren der Kapazität und vergleichen mit der bisherigen maximalen Kapazität
                capacity = float(capacity_attr.attrib['Value'])
                if capacity > max_capacity:
                    max_capacity = capacity
                    groesster_tank = tank
    
    # Wenn ein größerer Tank gefunden wurde
    if groesster_tank is not None:
        # Extrahieren der Tag-Bezeichnung des größten Tanks
        tank_tag = groesster_tank.attrib['TagName']
        
        # Extrahieren der Beschreibungen der Kammern im größten Tank
        tank_beschreibungen = [chamber.find(".//GenericAttribute[@Name='ChamberDescriptionAssignmentClass']").attrib.get('Value', '') for chamber in groesster_tank.iter('Equipment') if chamber.attrib.get('ComponentClass') == 'Chamber']
        
        return groesster_tank, tank_tag, tank_beschreibungen
    else:
        # None, wenn kein größerer Tank gefunden wurde
        return None, None

# Aufruf mit XML-Dokument
with open(dexpi_xml, 'r') as file:
    dexpi_xml_content = file.read()

groesster_tank, tank_tag, tank_beschreibungen = finde_groessten_tank(dexpi_xml_content)

# Ausgabe der Ergebnisse
#print("Größter Tank:", tank_tag)
#print("Beschreibungen der Kammern:", tank_beschreibungen)

def setze_anlagenart(tank_beschreibungen):
    # Überprüfen, ob 'Processing Chamber' in den Beschreibungen vorkommt
    if "Processing Chamber" in tank_beschreibungen:
        return "HBV-Anlage"
    # Überprüfen, ob 'Storage Chamber' in den Beschreibungen vorkommt
    elif "Storage Chamber" in tank_beschreibungen:
        return "Tanklager"
    else:
        # Fehlermeldung, wenn keine oder beide Beschreibungen vorhanden sind
        print("Fehler: Ungültige Anlagenkonfiguration. Anlagenart manuell eintragen.")
        return None

# Beispielaufruf
anlagenart = setze_anlagenart(tank_beschreibungen)

# Ausgabe der Ergebnisse
print(anlagenart)



HBV-Anlage


Angaben zu den wassergefährdenden Stoffen in der Anlage

In [20]:
### Ziel: Extrahieren der Stoffe ###

Angaben zur Gefährdungsstufe

In [21]:
### Ziel: Ermittlung der Gefährdungsstufe ###

Angaben zu Behältern

In [22]:
### Ziel: Extrahieren der Tank-Daten ###

# Initialisierung der Variable außerhalb der Funktion
material_of_construction = None

def tank_suche(dexpi_xml_path):
    # Einlesen des Inhalts der DEXPI-XML-Datei
    with open(dexpi_xml_path, 'r') as file:
        dexpi_xml_content = file.read()

    # Parsen der dexpi_xml und Erstellen des Wurzelelements
    root = ET.fromstring(dexpi_xml_content)
    tanks = []

    # Durchlaufen aller Equipment-Elemente mit ComponentClass='Tank'
    for equipment in root.findall(".//Equipment[@ComponentClass='Tank']"):
        # Extrahieren des Tag-Namens des Tanks
        tag_name = equipment.get('TagName', '')

        # Aussortieren von Tanks ohne Tag-Namen, die mit 'T' beginnen
        if not tag_name.startswith('T'):
            continue

        # Extrahieren des NominalCapacity-Elements
        nominal_capacity_element = equipment.find(".//GenericAttribute[@Name='NominalCapacity(Volume)']")
        
        # Überprüfen, ob NominalCapacity-Element vorhanden ist
        if nominal_capacity_element is not None:
            # Extrahieren des Wertes von NominalCapacity
            nominal_capacity_str = nominal_capacity_element.get('Value', '')

            # Überprüfen, ob NominalCapacity eine Zahl ist
            try:
                nominal_capacity = float(nominal_capacity_str)
            except ValueError:
                # Fehlermeldung ausgeben und zum nächsten Tank gehen
                print(f"Fehler: Nennvolumen für Tank mit Tanknummer '{tag_name}' ist keine Zahl.")
                continue
        else:
            # Fehlermeldung ausgeben und zum nächsten Tank gehen
            print(f"Fehler: Kein Nennvolumen für Tank mit Tanknummer '{tag_name}' vorhanden.")
            continue

        # Extrahieren der Kammern des Tanks
        chambers = []
        for chamber in equipment.findall(".//Equipment[@ComponentClass='Chamber']"):
            chamber_function = chamber.find(".//GenericAttribute[@Name='ChamberFunctionAssignmentClass']").get('Value', '')
            
            # Nur Kammern mit 'Process Chamber' oder 'Storage Chamber' berücksichtigen
            if chamber_function in ['Processing', 'Storage']:
                material_of_construction = chamber.find(".//GenericAttribute[@Name='MaterialOfConstructionCodeAssignmentClass']").get('Value', '')
                chambers.append({
                    'Function': chamber_function,
                    'MaterialOfConstructionCode': material_of_construction
                })

        # Zusammenstellen der Tankdaten und Hinzufügen zur Liste
        tank_data = {
            'TagName': tag_name,
            'NominalCapacity': nominal_capacity,
            'Chambers': chambers
        }
        tanks.append(tank_data)

    # Rückgabe der Liste von Tanks
    return tanks

# Aufrufen der Funktion mit der DEXPI-XML-Datei und Ausgabe der Ergebnisse
tanks = tank_suche(dexpi_xml)

for tank in tanks:
    print(f'TagName: {tank["TagName"]}, NominalCapacity: {tank["NominalCapacity"]}')
    
    for chamber in tank['Chambers']:
        print(f'  Chamber Function: {chamber["Function"]}, MaterialOfConstructionCode: {chamber["MaterialOfConstructionCode"]}')

TagName: T4750, NominalCapacity: 22.0
  Chamber Function: Processing, MaterialOfConstructionCode: 1.4306


KI-Anwendung Material
Quelle: https://python.langchain.com/docs/use_cases/chatbots/quickstart

In [23]:
# Importieren der benötigten Module
# import os

# Extrahieren des OpenAI-API-Schlüssels aus den Umgebungsvariablen oder direkt eintragen zwischen ""
# openai_api_key = os.getenv("OPENAI_API_KEY", "")

# Jetzt ist der OpenAI-API-Schlüssel verfügbar und kann im Code verwendet werden, der print unten nur zum Test
# print(f"OpenAI API Key: {openai_api_key}")

# Importieren der benötigten Module
# from langchain_openai import ChatOpenAI

# Initialisieren des ChatOpenAI-Objekts
# Mit der temperature kann man "herum experimentieren"
# chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1, openai_api_key=openai_api_key)


In [24]:
# Definition der Werkstoffbezeichnung, hier kein Automatismus
# Es können unterschiedliche Werkstoffe getestet werden
#werkstoff_bezeichnung = "1.4571"

# Verwendung der Variable in der Chat-Anfrage
#from langchain_core.messages import HumanMessage

#chat.invoke(
#    [
#        HumanMessage(
#            content=f'Setze die Sprache auf Deutsch. Du bist ein Assistent. Du hast eine Aufgabe. Ich gebe dir einen Werkstoff. Deine Aufgabe besteht darin, zuzuordnen, ob es sich beim gegebenen Werkstoff um ein Kunststoff, ein Metall oder einen anderes Material handelt. Versuche den gegebenen Werkstoff in eine der drei Kategorien: "Metall", "Kunststoff", "anderes Material" einzustufen. Antworte nur: "Metall", "Kunststoff", "anderes Material".  Der gesuchte Werkstoff ist: {werkstoff_bezeichnung}. Falls es sich dabei gar nicht um einen Werkstoff im engeren Sinn handelt sage "Error"'
#        )
#    ]
#)


Angaben zu den Rohrleitungen

In [25]:
### Ziel: Extrahieren der Rohrleitungen ###

Formular XML

In [26]:
### Ziel: Erstellen eines Formblatts 6.2 im .xml-Format ###
# Die anlagenspezifischen Werte welche ins Formblatt eingetragen werden stammen aus den oben ermittelten Variablen.

# Erstellen des Wurzelelements formblatt_6_2
formblatt_6_2 = ET.Element('formblatt_6_2')
xml_declaration = ET.ElementTree(formblatt_6_2)
xml_declaration_str = '<?xml version="1.0" encoding="UTF-8"?>'
formblatt_6_2.text = xml_declaration_str

def create_table(parent, table_name, heading, column_names, data):
    table = ET.SubElement(parent, f'tabelle_{table_name}')
    table_heading = ET.SubElement(table, 'ueberschrift')
    table_heading.text = heading

    columns = ET.SubElement(table, 'spalten')
    for column_name in column_names:
        column = ET.SubElement(columns, 'spalte')
        column.text = column_name

    table_data = ET.SubElement(table, 'daten')
    data_row = ET.SubElement(table_data, 'zeile')
    for value in data:
        value_element = ET.SubElement(data_row, 'wert')
        value_element.text = value

# Hier können die oben bestimmten Variablen eingesetzt werden 

# Kopftabelle
kopf_data = {
    'Antragsunterlage': 'Für immissionsschutzrechtliches Genehmigungsverfahren',
    'Anlage 1 / Formblatt 6.2': 'Detailangaben / Wassergefährdende Stoffe',
}

# Angaben zur Anlage
angaben_anlage_data = {
    'Anlagenart': anlagenart,
    'Anlagenbezeichnung': anlagenbezeichnung,
    'Anlagenumfang': anlagenumfang
}

# Angaben zu den wassergefährdenden Stoffen in der Anlage
angaben_stoffe_data = {
    'Bezeichnung': 'MNc',
    'Aggregatzustand': 'flüssig',
    'WGK': 'WGK3',
    'Volumen (m3) / Masse (t)': '22'
}

# Angaben zur Gefährdungsstufe
angaben_gefaehrdungsstufe_data = {
    'Maßgebendes Volumen': '22',
    'Maßgebende WGK': 'WGK3',
    'Gefährdungsstufe': 'D'
}

# Angaben zu Behältern
angaben_behaelter_data = {
    'Tanknummer': 'T4750',
    'Stoff': 'MNc',
    'einwandig / doppelwandig': 'einwandig',
    'Material': 'Metall',
    'Zulassung': 'DIN-Norm'
}

# Angaben zu Rohrleitungen
angaben_rohrleitungen_data = {
    'Bauart': 'Einwandige Rohrleitung',
    'oberirdisch / unterirdisch': 'oberirdisch',
    'Anzahl': '7',
    'Material': 'Metall',
    'Zulassung': 'DIN-Norm'
}


create_table(formblatt_6_2, 'kopf', 'Angaben zum Formblatt', kopf_data.keys(), kopf_data.values())
create_table(formblatt_6_2, 'angaben_anlage', 'Angaben zur Anlage', angaben_anlage_data.keys(), angaben_anlage_data.values())
create_table(formblatt_6_2, 'angaben_stoffe', 'Angaben zu den wassergefährdenden Stoffen', angaben_stoffe_data.keys(), angaben_stoffe_data.values())
create_table(formblatt_6_2, 'angaben_gefäaehrdungsstufe', 'Ermittlung der Gefährdungsstufe der Anlage nach §39 AwSV', angaben_stoffe_data.keys(), angaben_stoffe_data.values())
create_table(formblatt_6_2, 'angaben_gefäaehrdungsstufe', 'Ermittlung der Gefährdungsstufe der Anlage nach §39 AwSV', angaben_gefaehrdungsstufe_data.keys(), angaben_gefaehrdungsstufe_data.values())
create_table(formblatt_6_2, 'angaben_behaelter', 'Angaben zu Behältern', angaben_behaelter_data.keys(), angaben_behaelter_data.values())
create_table(formblatt_6_2, 'angaben_rohrleitungen', 'Angaben zu Rohrleitungen', angaben_rohrleitungen_data.keys(), angaben_rohrleitungen_data.values())

# Drucken des XML-Elements
xml_str = ET.tostring(formblatt_6_2, encoding='utf-8').decode()
print(xml_str)




<formblatt_6_2>&lt;?xml version="1.0" encoding="UTF-8"?&gt;<tabelle_kopf><ueberschrift>Angaben zum Formblatt</ueberschrift><spalten><spalte>Antragsunterlage</spalte><spalte>Anlage 1 / Formblatt 6.2</spalte></spalten><daten><zeile><wert>Für immissionsschutzrechtliches Genehmigungsverfahren</wert><wert>Detailangaben / Wassergefährdende Stoffe</wert></zeile></daten></tabelle_kopf><tabelle_angaben_anlage><ueberschrift>Angaben zur Anlage</ueberschrift><spalten><spalte>Anlagenart</spalte><spalte>Anlagenbezeichnung</spalte><spalte>Anlagenumfang</spalte></spalten><daten><zeile><wert>HBV-Anlage</wert><wert>C01V01-HEX.EX03</wert><wert>Zentrifugalpumpe, Rohrleitungssystem, Kolbenpumpe, Plattenwärmetauscher, Behälter, Rohrbündelwärmetauscher</wert></zeile></daten></tabelle_angaben_anlage><tabelle_angaben_stoffe><ueberschrift>Angaben zu den wassergefährdenden Stoffen</ueberschrift><spalten><spalte>Bezeichnung</spalte><spalte>Aggregatzustand</spalte><spalte>WGK</spalte><spalte>Volumen (m3) / Masse (

Formular PDF

PDF mit latex // Pfad auf Desktop --> ändern?

In [27]:
### Ziel: Erstellen eines Formblatts 6.2 im .tex-Format ###
# Die anlagenspezifischen Werte welche ins Formblatt eingetragen werden stammen aus den oben ermittelten Variablen.

import os

# LaTeX-Template als String in einer Zelle
latex_template = r"""
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{enumitem}

\begin{document}

\section*{Angaben zum Formblatt}

\subsection*{Antragsunterlage}

\begin{itemize}[label=--, left=0pt, itemsep=0pt]
    \item \textbf{Für immissionsschutzrechtliches Genehmigungsverfahren}
    \item \textbf{Detailangaben / Wassergefährdende Stoffe}
\end{itemize}

\subsection*{Angaben zur Anlage}

\begin{tabular}{|l|l|l|}
    \hline
    \textbf{Anlagenart} & \textbf{Anlagenbezeichnung} & \textbf{Anlagenumfang} \\
    \hline
    \VAR{anlagenart} & \VAR{anlagenbezeichnung} & \VAR{anlagenumfang} \\
    \hline
\end{tabular}

\subsection*{Angaben zu den wassergefährdenden Stoffen}

\begin{tabular}{|l|l|l|l|}
    \hline
    \textbf{Bezeichnung} & \textbf{Aggregatzustand} & \textbf{WGK} & \textbf{Volumen (m3) / Masse (t)} \\
    \hline
    \VAR{stoff_bezeichnung} & \VAR{stoff_aggregatzustand} & \VAR{stoff_wgk} & \VAR{stoff_volumen_masse} \\
    \hline
\end{tabular}

\subsection*{Ermittlung der Gefährdungsstufe der Anlage nach §39 AwSV}

\begin{tabular}{|l|l|l|}
    \hline
    \textbf{Maßgebendes Volumen} & \textbf{Maßgebende WGK} & \textbf{Gefährdungsstufe} \\
    \hline
    \VAR{anlage_volumen} & \VAR{anlage_wgk} & \VAR{anlage_gefaehrdungsstufe} \\
    \hline
\end{tabular}

\subsection*{Angaben zu Behältern}

\begin{tabular}{|l|l|l|l|l|}
    \hline
    \textbf{Tanknummer} & \textbf{Stoff} & \textbf{einwandig / doppelwandig} & \textbf{Material} & \textbf{Zulassung} \\
    \hline
    \VAR{tank_nummer} & \VAR{tank_stoff & \VAR{tank_wandig} & \VAR{tank_material} \VAR{tank_zulassung} \\
    \hline
\end{tabular}

\subsection*{Angaben zu Rohrleitungen}

\begin{tabular}{|l|l|l|l|l|}
    \hline
    \textbf{Bauart} & \textbf{oberirdisch / unterirdisch} & \textbf{Anzahl} & \textbf{Material} & \textbf{Zulassung} \\
    \hline
    \VAR{rl_bauart} & \VAR{rl_irdisch} & \VAR{rl_anzahl} & \VAR{rl_material} \VAR{rl_zulassung} \\
    \hline
\end{tabular}

\end{document}

"""

# Ersetzen der Variablen im Template
latex_code = latex_template.replace("\\VAR{anlagenbezeichnung}", anlagenbezeichnung)
latex_code = latex_code.replace("\\VAR{anlagenart}", anlagenart)
latex_code = latex_code.replace("\\VAR{anlagenumfang}", anlagenumfang)


# Pfad auf Desktop 
desktop_path = os.path.join("/Users/millerfl/Desktop", "formular_6_2_auto.tex") #Datei lokal in Ordner legen + Pfad entfernen #Evtl auch Version in Collab ablegen

# Speichern des LaTeX-Codes in einer .tex-Datei auf dem Desktop
with open(desktop_path, "w") as tex_file:
    tex_file.write(latex_code)

Back-Up

PDF-Test simpel

In [28]:
#from reportlab.pdfgen import canvas
#from reportlab.lib.pagesizes import letter

# Pfad zur PDF-Datei
#pdf_path = "/Users/millerfl/Desktop/test.pdf"

# Wort, das in die PDF geschrieben werden soll
#word_to_write = "Hello"

# Funktion zum Erstellen der PDF mit einem Wort
#def create_pdf(word, pdf_path):
    #pdf_canvas = canvas.Canvas(pdf_path, pagesize=letter)
    #pdf_canvas.drawString(100, 750, word)  # Position und Wort anpassen
    #pdf_canvas.save()

# Erstellen der PDF
#create_pdf(word_to_write, pdf_path)


PDF-Test erweitert

In [29]:
# from reportlab.pdfgen import canvas
# from reportlab.lib.pagesizes import letter
# from reportlab.lib import colors
# from reportlab.platypus import Table, TableStyle
# import os
# import xml.etree.ElementTree as ET

# # Pfad zur PDF-Datei
# pdf_path = "/Users/millerfl/Desktop/formblatt_6_2.pdf"

# # Beispiel-XML-String (später ersetzen durch tatsächlichen XML-String)
# xml_str = """
# <formblatt_6_2>
#     <tabelle_angaben_anlage>
#         <daten>
#             <zeile>
#                 <wert>T1234</wert>
#                 <wert>ABC</wert>
#                 <wert>einwandig</wert>
#                 <wert>Metall</wert>
#                 <wert>DIN-Norm</wert>
#             </zeile>
#         </daten>
#     </tabelle_angaben_anlage>
# </formblatt_6_2>
# """

# # Erstellen des XML-Elements aus dem String
# formblatt_6_2 = ET.fromstring(xml_str)

# # Funktion zum Erstellen der PDF mit reportlab und alternativem Ansatz
# def create_pdf_alternative(xml_element, pdf_path):
#     try:
#         pdf_canvas = canvas.Canvas(pdf_path, pagesize=letter)
#         table_data = []

#         for child in xml_element.findall(".//zeile"):
#             row_data = [subchild.text for subchild in child]
#             table_data.append(row_data)

#         table_height = len(table_data) * 10
#         table_width = len(table_data[0]) * 100

#         # Setzen der Startposition für die Tabelle
#         x_offset, y_offset = 100, 750

#         # Erstellen der Table-Instanz und Hinzufügen der Daten
#         pdf_table = Table(table_data, colWidths=100, rowHeights=10)

#         # Definieren des Stils der Tabelle
#         style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
#                             ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
#                             ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
#                             ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
#                             ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
#                             ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
#                             ('GRID', (0, 0), (-1, -1), 1, colors.black)])

#         pdf_table.setStyle(style)

#         # Zeichnen der Tabelle auf den Canvas
#         pdf_table.wrapOn(pdf_canvas, 0, 0)
#         pdf_table.drawOn(pdf_canvas, x_offset, y_offset - table_height)

#         pdf_canvas.save()
#         print("PDF erfolgreich erstellt!")
#     except Exception as e:
#         print(f"Fehler beim Erstellen der PDF: {e}")

# # Erstellen des PDFs
# create_pdf_alternative(formblatt_6_2, pdf_path)


PDF-Test mit XSLT

In [30]:
# GitHub Raw-URL für die XSLT
github_url_xslt = 'https://github.com/FloT10/MillerFlorinBA/blob/c60ce79783cdf3a9879b88448c7d054692c50ada/transform.xslt'

# Herunterladen der XSLT-Datei von der GitHub-URL.
response = requests.get(github_url_xslt)

# Speichern der heruntergeladenen XSLT-Datei lokal im Textmodus ('w').
with open('/tmp/transform.xslt', 'w') as file:
    file.write(response.text)

# Definieren eines Dateipfades zur lokal gespeicherten XSLT-Datei zur weiteren Verwendung.
transform_xslt_path = "/tmp/transform.xslt"

# Öffne die Datei und lese ihren Inhalt.
with open(transform_xslt_path, 'r') as file:
    transform_xslt_content = file.read()

# Gib den Inhalt der XSLT-Datei aus.
print(transform_xslt_content)

# Da Probleme mit Pfad vorübergehende Alternative
xslt_path = "/Users/millerfl/Desktop/transform.xslt"

{"payload":{"allShortcutsEnabled":false,"fileTree":{"":{"items":[{"name":".vscode","path":".vscode","contentType":"directory"},{"name":"C01V01-HEX.EX03.xml","path":"C01V01-HEX.EX03.xml","contentType":"file"},{"name":"LICENSE","path":"LICENSE","contentType":"file"},{"name":"README.md","path":"README.md","contentType":"file"},{"name":"formblatt_6.2.xml","path":"formblatt_6.2.xml","contentType":"file"},{"name":"prototyp.ipynb","path":"prototyp.ipynb","contentType":"file"},{"name":"transform.xslt","path":"transform.xslt","contentType":"file"}],"totalCount":7}},"fileTreeProcessingTime":2.1320900000000003,"foldersToFetch":[],"repo":{"id":733371200,"defaultBranch":"main","name":"MillerFlorinBA","ownerLogin":"FloT10","currentUserCanPush":false,"isFork":false,"isEmpty":false,"createdAt":"2023-12-19T07:14:43.000Z","ownerAvatar":"https://avatars.githubusercontent.com/u/143116830?v=4","public":true,"private":false,"isOrgOwned":false},"symbolsExpanded":false,"treeExpanded":true,"refInfo":{"name":"c

In [31]:
import os

# Aktuelles Arbeitsverzeichnis anzeigen
current_directory = os.getcwd()
print("Aktuelles Arbeitsverzeichnis:", current_directory)


Aktuelles Arbeitsverzeichnis: /


In [32]:
from lxml import etree

#Zurückgreifen auf vorhanden String
print(xml_str)

# Laden der XSLT-Datei
xslt_root = etree.parse(xslt_path)
transform = etree.XSLT(xslt_root)
xml_tree = etree.fromstring(xml_str)

result = transform(xml_tree)

#with open("output.html", "wb") as f:
#    f.write(result)

<formblatt_6_2>&lt;?xml version="1.0" encoding="UTF-8"?&gt;<tabelle_kopf><ueberschrift>Angaben zum Formblatt</ueberschrift><spalten><spalte>Antragsunterlage</spalte><spalte>Anlage 1 / Formblatt 6.2</spalte></spalten><daten><zeile><wert>Für immissionsschutzrechtliches Genehmigungsverfahren</wert><wert>Detailangaben / Wassergefährdende Stoffe</wert></zeile></daten></tabelle_kopf><tabelle_angaben_anlage><ueberschrift>Angaben zur Anlage</ueberschrift><spalten><spalte>Anlagenart</spalte><spalte>Anlagenbezeichnung</spalte><spalte>Anlagenumfang</spalte></spalten><daten><zeile><wert>HBV-Anlage</wert><wert>C01V01-HEX.EX03</wert><wert>Zentrifugalpumpe, Rohrleitungssystem, Kolbenpumpe, Plattenwärmetauscher, Behälter, Rohrbündelwärmetauscher</wert></zeile></daten></tabelle_angaben_anlage><tabelle_angaben_stoffe><ueberschrift>Angaben zu den wassergefährdenden Stoffen</ueberschrift><spalten><spalte>Bezeichnung</spalte><spalte>Aggregatzustand</spalte><spalte>WGK</spalte><spalte>Volumen (m3) / Masse (

OSError: Error reading file '/Users/millerfl/Desktop/transform.xslt': failed to load external entity "/Users/millerfl/Desktop/transform.xslt"

In [ ]:
import pdfkit

pdfkit.from_file("output.html", "output.pdf")


OSError: No such file: output.html

Test auf Zugriff 
(Ausführen im Terminal)
ls -l /Users/millerfl/Desktop/formblatt_6_2.pdf

Sonstiges

In [ ]:
# Später für Aufzählung
# Funktion zum Laden und Extrahieren von Daten aus der DEXPI-Datei.
def lade_daten(dexpi_xml):
    try:
        # Lade das XML-Dokument und erhalte die Wurzel des Baums.
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahiere die Anlagenbezeichnung aus dem XML-Baum.
        anlagenbezeichnung = root.find('.//Drawing').get('Name')

        # Extrahiere ComponentClass aus allen Equipment- und PipingNetworkSystem-Elementen.
        equipment_elements = root.findall('.//Equipment')
        piping_network_system_elements = root.findall('.//PipingNetworkSystem')

        component_classes = []

        for equipment_element in equipment_elements:
            component_class = equipment_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            erlaubte_component_classes = ['Tank', 'ReciprocatingPump', 'CentrifugalPump', 'PlateAndShellHeatExchanger', 'ShellAndTubeHeatExchanger']
            if component_class in erlaubte_component_classes:
                component_classes.append(component_class)

        for piping_network_system_element in piping_network_system_elements:
            component_class = piping_network_system_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            erlaubte_component_classes = ['PipingNetworkSystem']
            if component_class in erlaubte_component_classes:
                component_classes.append(component_class)

        # Überprüfe, ob mindestens eine gültige ComponentClass gefunden wurde.
        if not component_classes:
            raise ValueError("Keine gültige ComponentClass gefunden.")

        # Weitere Werte können entsprechend extrahiert werden.

        # Entferne Duplikate und erstelle ein eindeutiges Set von Komponentenklassen.
        unique_component_classes = set(component_classes)

        # Erstelle ein Dictionary mit dem Namen Anlagenumfang, in dem jede ComponentClass nur einmal vorkommt.
        Anlagenumfang = {klass: {} for klass in unique_component_classes}

        # Gib ein Dictionary mit den extrahierten Werten zurück.
        return {
            'anlagenbezeichnung': anlagenbezeichnung,
            'component_classes': component_classes,
            'Anlagenumfang': Anlagenumfang,
            # Weitere extrahierte Werte können hier hinzugefügt werden.
        }

    except Exception as e:
        # Falls ein Fehler auftritt, gib eine Fehlermeldung aus und gibt None zurück.
        print(f"Fehler beim Laden und Extrahieren der Daten: {e}")
        return None

# Beispielaufruf der Funktion und Ausgabe der extrahierten Daten.
daten = lade_daten(dexpi_xml)
print(daten)


{'anlagenbezeichnung': 'C01V01-HEX.EX03', 'component_classes': ['Tank', 'ReciprocatingPump', 'CentrifugalPump', 'ShellAndTubeHeatExchanger', 'PlateAndShellHeatExchanger', 'Tank', 'ReciprocatingPump', 'CentrifugalPump', 'ShellAndTubeHeatExchanger', 'PlateAndShellHeatExchanger', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem'], 'Anlagenumfang': {'PipingNetworkSystem': {}, 'ReciprocatingPump': {}, 'PlateAndShellHeatExchanger': {}, 'ShellAndTubeHeatExchanger': {}, 'Tank': {}, 'CentrifugalPump': {}}}


In [ ]:
import requests
import xml.etree.ElementTree as ET

# GitHub Raw-URL für die DEXPI-R&I Datei
github_url_dexpi = 'https://raw.githubusercontent.com/FloT10/MillerFlorinBA/main/C01V01-HEX.EX03.xml'
# GitHub Raw-URL für die Formblatt-Vorlage
github_url_formblatt = 'https://raw.githubusercontent.com/FloT10/MillerFlorinBA/main/formblatt_6.2.xml'

# Dateien lokal speichern
def lade_und_speichere(url, lokal_path):
    response = requests.get(url)
    with open(lokal_path, 'w') as file:
        file.write(response.text)

# Laden und Speichern der DEXPI-R&I Datei
dexpi_xml = '/tmp/C01V01-HEX.EX03.xml'
lade_und_speichere(github_url_dexpi, dexpi_xml)

# Laden und Speichern der Formblatt-Vorlage
formblatt_vorlage = '/tmp/formblatt_6.2.xml'
lade_und_speichere(github_url_formblatt, formblatt_vorlage)

# Funktion zum Laden und Extrahieren der Werte aus dem DEXPI-R&I
def lade_daten(dexpi_xml):
    try:
        # Laden des DEXPI-R&I Dokuments
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahieren der Werte, wie benötigt
        anlagenbezeichnung = root.find('.//Drawing').get('Name')
        # Weitere Werte entsprechend extrahieren

        # Kontrollausgabe der Anlagenbezeichnung
        print(f"Anlagenbezeichnung zur Kontrolle: {anlagenbezeichnung}")

        return {
            'anlagenbezeichnung': anlagenbezeichnung,
            # Weitere extrahierte Werte hinzufügen
        }

    except Exception as e:
        print(f"Fehler beim Laden und Extrahieren der Daten aus DEXPI-R&I: {e}")
        return None

# Funktion zum Befüllen der Formblatt-Vorlage mit den extrahierten DEXPI-R&I Daten
def befuellen_und_speichern(formblatt_vorlage, formblatt_befüllt, daten):
    try:
        tree = ET.parse(formblatt_vorlage)
        root = tree.getroot()

        # Befülle die Vorlage mit den extrahierten Daten
        root.find('.//anlagenbezeichnung').text = daten['anlagenbezeichnung']
        # Weitere Werte in der Vorlage entsprechend befüllen

        # Speichere die befüllte Vorlage als neue Datei
        tree.write(formblatt_befüllt)

        print(f"Das Formblatt wurde erfolgreich mit den Daten befüllt und unter {formblatt_befüllt} gespeichert.")

    except Exception as e:
        print(f"Fehler beim Befüllen und Speichern des Formblatts: {e}")

# Lade Daten aus der DEXPI-R&I Datei
daten_dexpi = lade_daten(dexpi_xml)

# Lade die Formblatt-Vorlage
formblatt_befüllt= '/tmp/Ziel/Formblatt_befüllt.xml'

# Befülle die Formblatt-Vorlage und speichere das Ergebnis
befuellen_und_speichern(formblatt_vorlage, formblatt_befüllt, daten_dexpi)


Anlagenbezeichnung zur Kontrolle: C01V01-HEX.EX03
Fehler beim Befüllen und Speichern des Formblatts: mismatched tag: line 233, column 2
